In [3]:
# Importowanie potrzebnych bibliotek
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Pinecone
from tqdm.autonotebook import tqdm
import pinecone 
import openai
import os
import re

# Zdefiniowanie zmiennej 'directory' przechowującej ścieżkę do folderu z danymi
directory = 'data'

def load_docs(directory):
  loader = DirectoryLoader(directory)
  documents = loader.load()
  return documents

documents = load_docs(directory)

# Wyświetlenie liczby załadowanych dokumentów
len(documents)

2

In [5]:
# Ustawienie zmiennej środowiskowej 'OPENAI_API_KEY' na konkretny klucz API
os.environ["OPENAI_API_KEY"] = " "

# Zdefiniowanie modelu do embeddingów
embeddings = OpenAIEmbeddings(model_name="text-embedding-ada-002")

In [6]:
for document in documents:
    # Usunięcie myślników na końcu linii i połączenie słów
    document.page_content = re.sub(r'-\n', '', document.page_content)
    # Usunięcie numerów stron (enter, liczby, enter)
    document.page_content = re.sub('\n\d+\n', '\n', document.page_content)
    # Usunięcie wszystkich enterów
    document.page_content = re.sub('\n', '', document.page_content)

In [12]:
# funkcja split_docs, która dzieli dokumenty na chunki
def split_docs(documents,chunk_size=2000,chunk_overlap=150):
  text_splitter = SentenceTransformersTokenTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(documents)
print(len(docs))

1742


In [14]:
# Wyświetlenie zawartości wybranego chunka
print(docs[90].page_content)

interdyscyplinarny charakter nauki o organizacji i zarzadzaniu po - woduje, ze obie centralne kategorie zawarte w nazwie dyscypliny sa roznie de€niowane, interpretowane. szczegolnie dotyczy to termi - nu zarzadzania i innych bliskoznacznych terminow, np. kierowania, administrowania itp. interdyscyplinarny charakter nauki o organizacji i zarzadzaniu w naturalny sposob generował wielosc historycznych i wspołczesnych nurtow i szkoł zarzadzania. roznorodnosc obiektow badawczych w sensie ich wielkosci, struktur i funkcji, na ktorych opi - sywane sa systemy zarzadzania, coraz wieksza turbulencja otoczenia organizacji, wpływajaca na zmiany tresci i formy zarzadzania nimi, dodatkowo komplikuje identy€kacje kategorii epistemologicznych na adekwatnym nauce poziomie ogolnosci. brak precyzyjnych i jedno - znacznie rozumianych podstawowych kategorii w nauce rodzi podejrze - nia o brak wyraznie wyodrebnionego przedmiotu badan, a jej niechet - ni przynajmnie


In [15]:
# Inicjalizacja Pinecone, platformy do zarządzania bazami wektorowymi
pinecone.init(
    api_key=" ",  # Klucz API
    environment=" "  # Nazwa środowiska, która jest dostępna w konsoli obok klucza API
)

index_name = " " # Nazwa indeksu

# Tworzenie indeksu w Pinecone z dokumentów i osadzeń (embeddings), oraz nadanie mu nazwy
index = Pinecone.from_documents(docs, embeddings, index_name=index_name)

In [16]:
# Definicja funkcji get_similiar_docs, która zwraca chunki podobne do zadanego zapytania
def get_similiar_docs(query,k=1,score=False):
  if score:
    similar_docs = index.similarity_search_with_score(query,k=k)
  else:
    similar_docs = index.similarity_search(query,k=k)
  return similar_docs

In [17]:
# Ustawienie zmiennej 'query' na tekst zapytania
query = "Czym są systemy ERP?"

# Wywołanie funkcji get_similiar_docs z zapytaniem 'query'
similar_docs = get_similiar_docs(query)
similar_docs

[Document(page_content='##p – definicja i charakterystykajak podkresla z. j. klonowski [ 1995, s. 92 ], w literaturze przedmiotu nie ma ujed - noliconej definicji systemu erp ( ang. enterprise resource planning ). mozna tez spotkac wiele roznych prob zdefiniowania tych systemow, nierzadko znaczaco od sie - bie odbiegajacych i akcentujacych bardziej odmienne ich aspekty. na potrzeby prezentowanego opracowania mozna przyjac nastepujaca definicjesystemow erp : zintegrowanym systemem informatycznym wspomagajacym zarzadzanie klasy erp ( w skrocie : systemem erp ) jest oprogramowanie standardowe oferowane w formie pakietow, majace na celu wspomaganie zarzadzania przedsiebiorstwem. jego funk - cjonalnosc w całosci obejmuje standard mrp ii w wersji rozwinietej, ma lub jest zin - tegrowane z modułami umozliwiajacymi obsługe procesow z obszaru finansow i control - lingu oraz zarzadzania zasobami ludzkimi i moze byc w prosty sposob zintegrowane z innymi modułami funkcjonalnymi oraz system', metad